In [ ]:
!pip install --upgrade -q git+https://github.com/keras-team/keras-cv
!pip install --upgrade -q git+https://github.com/soumik12345/wandb-addons wandb
!pip install -q namex
!apt install python3.10-venv
!git clone https://github.com/keras-team/keras-core.git && cd keras-core
!python pip_build.py --install

In [1]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import keras_cv

import keras_core as keras
from keras_core import ops

import numpy as np

import wandb
from wandb_addons.keras.detection import log_predictions_to_wandb

2023-08-16 10:51:16.984642: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Using TensorFlow backend


In [2]:
wandb.init(project="keras-cv-callbacks", job_type="detection")

config = wandb.config
config.model_name = "retinanet_resnet50_pascalvoc"
config.image_size = 640
config.class_ids = [
    "Aeroplane",
    "Bicycle",
    "Bird",
    "Boat",
    "Bottle",
    "Bus",
    "Car",
    "Cat",
    "Chair",
    "Cow",
    "Dining Table",
    "Dog",
    "Horse",
    "Motorbike",
    "Person",
    "Potted Plant",
    "Sheep",
    "Sofa",
    "Train",
    "Tvmonitor",
    "Total",
]

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: geekyrakshit. Use `wandb login --relogin` to force relogin


In [3]:
pretrained_model = keras_cv.models.RetinaNet.from_preset(
    config.model_name, bounding_box_format="xywh"
)

2023-08-16 10:51:24.504104: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-16 10:51:24.510325: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-16 10:51:24.510560: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
filepath_1 = keras.utils.get_file(origin="https://i.imgur.com/gCNcJJI.jpg")
filepath_2 = keras.utils.get_file(origin="https://i.imgur.com/M8LR4fz.png")

image_1 = keras.utils.load_img(filepath_1)
image_1 = np.array(image_1)
image_2 = keras.utils.load_img(filepath_2)
image_2 = np.array(image_2)

In [5]:
inference_resizing = keras_cv.layers.Resizing(
    config.image_size,
    config.image_size,
    pad_to_aspect_ratio=True,
    bounding_box_format="xywh"
)
image_1_resized = inference_resizing([image_1])
image_2_resized = inference_resizing([image_2])
image_batch = ops.concatenate([image_1_resized, image_2_resized], axis=0)
image_batch.shape

TensorShape([2, 640, 640, 3])

In [6]:
class_mapping = dict(zip(range(len(config.class_ids)), config.class_ids))

y_pred = pretrained_model.predict(image_batch)

log_predictions_to_wandb(image_batch, y_pred, class_mapping)

wandb.finish()

2023-08-16 10:51:55.257876: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55cea642a1b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-16 10:51:55.257934: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2023-08-16 10:51:55.458641: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-16 10:51:56.751972: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-16 10:52:05.350798: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step


  0%|          | 0/2 [00:00<?, ?it/s]

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
